# 线性回归的简介实现

> 通过使用深度学习框架来简洁实现 线性回归模型 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2., -3.5])
true_b = 3.1

features, labels = d2l.synthetic_data(true_w, true_b, 1000)

### 调用框架现有的API读取数据

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch的数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train) 

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [5]:
next(iter(data_iter))

[tensor([[ 1.0626, -0.4919],
         [-0.3683, -0.8259],
         [ 0.8332,  1.4517],
         [ 1.1360, -0.3633],
         [-0.0521,  1.2288],
         [ 2.1743,  0.6291],
         [ 1.9912,  0.1031],
         [-0.3643,  0.2114],
         [ 0.9401, -0.7548],
         [-0.5997, -1.2991]]),
 tensor([[ 6.9416],
         [ 5.2671],
         [-0.3094],
         [ 6.6356],
         [-1.2929],
         [ 5.2572],
         [ 6.7202],
         [ 1.6180],
         [ 7.6273],
         [ 6.4429]])]

### 使用框架预定义好的层

In [6]:
from torch import nn

In [7]:
net = nn.Sequential(nn.Linear(2, 1))

### 初始化模型参数 

In [8]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

### 计算均方误差使用 MSELoss类， 也称L2范数

In [9]:
loss = nn.MSELoss()

### 实例化SGD实例

In [10]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### 训练

In [11]:
num_epochs = 3

for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()  # 参数更新
    l = loss(net(features), labels)
    print(f"epoch {epoch + 1}, loss {l:f}")

epoch 1, loss 0.000233
epoch 2, loss 0.000094
epoch 3, loss 0.000095


In [12]:
net.parameters()

<generator object Module.parameters at 0x7fa4f03e8eb0>